# Sensitivity Analysis

After finding the optimal decision strategy, it's crucial to understand how sensitive our results are to changes in the input parameters. **Sensitivity analysis** helps identify which uncertainties have the greatest impact on the expected utility and, consequently, on the optimal decisions.

This notebook is going to be focused on generating plots for sensitivity analysis of the Oil Decision Problem.

In [1]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import pyagrum as grum
import pyagrum.lib.notebook as gnb
from pyagrum import InfluenceDiagram

## 1 - Baseline

### 1.1 - Influence diagram structure

In [2]:
influence_diagram = InfluenceDiagram()

Q = influence_diagram.addChanceNode(grum.LabelizedVariable("Q", "Q", 0).addLabel('high').addLabel('medium').addLabel('low'))
R = influence_diagram.addChanceNode(grum.LabelizedVariable("R", "R", 0).addLabel('pass').addLabel('fail').addLabel('no_results'))
T = influence_diagram.addDecisionNode(grum.LabelizedVariable("T", "T", 0).addLabel('do').addLabel('not_do'))
B = influence_diagram.addDecisionNode(grum.LabelizedVariable("B", "B", 0).addLabel('buy').addLabel('not_buy'))
U = influence_diagram.addUtilityNode(grum.LabelizedVariable("U", "U", 0).addLabel('utility'))

influence_diagram.addArc("T", "R")
influence_diagram.addArc("T", "B") # memory arc
influence_diagram.addArc("T", "U")
influence_diagram.addArc("R", "B")
influence_diagram.addArc("B", "U")
influence_diagram.addArc("Q", "R")
influence_diagram.addArc("Q", "U")

gnb.sideBySide(influence_diagram, captions=["Oil field influence diagram"])

### 1.2 - Influence diagram parameters

In [3]:
q_cpt = pd.DataFrame({
    "Q": ["high", "medium", "low"],
    "Probability": [0.35, 0.45, 0.2]
})
# Set Q values as index for easier access
q_cpt = q_cpt.set_index('Q')

r_cpt = pd.DataFrame({
    "R | Q": ["pass", "fail"],
    "high": [0.95, 0.05],
    "medium": [0.7, 0.3],
    "low": [0.15, 0.85]
})
# Set R | Q values as index for easier access
r_cpt = r_cpt.set_index('R | Q')

u_table = pd.DataFrame(
    [
        ['do', 'buy', 'high', 1250],
        ['do', 'buy', 'medium', 630],
        ['do', 'buy', 'low', 0],
        ['do', 'not_buy', '-', 350],
        ['not_do', 'buy', 'high', 1280],
        ['not_do', 'buy', 'medium', 660],
        ['not_do', 'buy', 'low', 30],
        ['not_do', 'not_buy', '-', 380],
    ],
    columns=['T', 'B', 'Q', 'U']
)

# Q probabilities
influence_diagram.cpt(Q)[:] = q_cpt['Probability'].to_list()

# R probabilities
for q in ["high", "medium", "low"]:
    pass_prob = r_cpt.loc['pass', q]
    fail_prob = r_cpt.loc['fail', q]

    influence_diagram.cpt(R)[{"Q": q, "T": "do"}] = [pass_prob, fail_prob, 0.0]
    influence_diagram.cpt(R)[{"Q": q, "T": "not_do"}] = [0.0, 0.0, 1.0]

# U utility values
influence_diagram.utility(U)[{"T": "do", "B": "buy"}] = np.array(
    [u_table.iloc[0, 3], u_table.iloc[1, 3], u_table.iloc[2, 3]])[:, np.newaxis]
influence_diagram.utility(U)[{"T": "do", "B": "not_buy"}] = np.array(
    [u_table.iloc[3, 3]] * 3)[:, np.newaxis]
influence_diagram.utility(U)[{"T": "not_do", "B": "buy"}] = np.array(
    [u_table.iloc[4, 3], u_table.iloc[5, 3], u_table.iloc[6, 3]])[:, np.newaxis]
influence_diagram.utility(U)[{"T": "not_do", "B": "not_buy"}] = np.array(
    [u_table.iloc[7, 3]] * 3)[:, np.newaxis]

In [4]:
q_cpt

,Probability
Q,
high,0.35
medium,0.45
low,0.20


In [5]:
r_cpt

,high,medium,low
R | Q,,,
pass,0.95,0.7,0.15
fail,0.05,0.3,0.85


### 1.3 - Baseline Optimal decision

In [6]:
inference_engine = grum.ShaferShenoyLIMIDInference(influence_diagram)
inference_engine.makeInference()
meu_result = inference_engine.MEU()
baseline_meu = meu_result['mean']

print(inference_engine.optimalDecision("B").argmax())
print(inference_engine.optimalDecision("T").argmax())

print(baseline_meu)

([{'B': 0, 'R': 0}, {'B': 0, 'R': 1}, {'B': 0, 'R': 2}], 1.0)
([{'T': 1}], 1.0)
751.0


## 2 - Tornado plot

### 2.1 - What is a Tornado Diagram?

A **tornado diagram** is a visualization tool used in sensitivity analysis that shows:

- **Which variables have the most impact** on the expected utility
- **The direction of impact** (positive or negative) when parameters change
- **The magnitude of sensitivity** to help prioritize where to focus attention

The diagram gets its name from its characteristic shape: variables are sorted by impact magnitude (largest at top, smallest at bottom), creating a tornado-like appearance.

### 2.2 - How Tornado Analysis Works

The process involves a **one-at-a-time analysis**:

1. **Establish baseline**: Calculate expected utility with current parameter values
2. **Vary each parameter individually**: Change one parameter by a fixed percentage (e.g., ±10%) while keeping all others at baseline
3. **Calculate impact**: Record how the expected utility changes for each parameter variation
4. **Visualize results**: Create horizontal bars showing the range of impact for each variable

### 2.3 - Implementation

> Even though tornado plots can be used for anything. We are going to focus on utility tables because it is usually where they are most common, or on variables that affect those utilities directly. In this case it would be the test cost.

In [7]:
def update_utility_and_calculate_meu_with_decisions(u_table_modified):
    """
    Update utility values in the influence diagram and calculate MEU with optimal decisions
    
    Parameters:
    -----------
    u_table_modified : pd.DataFrame
        Modified utility table
        
    Returns:
    --------
    tuple : (MEU, optimal_decisions_dict)
    """
    # Store original utility values
    original_utilities = {}
    original_utilities["do_buy"] = influence_diagram.utility(U)[{"T": "do", "B": "buy"}].copy()
    original_utilities["do_not_buy"] = influence_diagram.utility(U)[{"T": "do", "B": "not_buy"}].copy()
    original_utilities["not_do_buy"] = influence_diagram.utility(U)[{"T": "not_do", "B": "buy"}].copy()
    original_utilities["not_do_not_buy"] = influence_diagram.utility(U)[{"T": "not_do", "B": "not_buy"}].copy()
    
    try:
        # Update utility values
        influence_diagram.utility(U)[{"T": "do", "B": "buy"}] = np.array(
            [u_table_modified.iloc[0, 3], u_table_modified.iloc[1, 3], u_table_modified.iloc[2, 3]])[:, np.newaxis]
        influence_diagram.utility(U)[{"T": "do", "B": "not_buy"}] = np.array(
            [u_table_modified.iloc[3, 3]] * 3)[:, np.newaxis]
        influence_diagram.utility(U)[{"T": "not_do", "B": "buy"}] = np.array(
            [u_table_modified.iloc[4, 3], u_table_modified.iloc[5, 3], u_table_modified.iloc[6, 3]])[:, np.newaxis]
        influence_diagram.utility(U)[{"T": "not_do", "B": "not_buy"}] = np.array(
            [u_table_modified.iloc[7, 3]] * 3)[:, np.newaxis]
        
        # Calculate MEU and optimal decisions
        temp_inference = grum.ShaferShenoyLIMIDInference(influence_diagram)
        temp_inference.makeInference()
        meu_result = temp_inference.MEU()
        meu = meu_result['mean']
        
        # Get optimal decisions
        optimal_T = temp_inference.optimalDecision("T").argmax()
        optimal_B_tensor = temp_inference.optimalDecision("B")
        
        # For T decision (simple decision)
        t_decision = 'do' if optimal_T[0][0]['T'] == 0 else 'not_do'
        
        # For B decision (depends on R), extract the policy
        b_instantiations, _ = optimal_B_tensor.argmax()
        b_policy = {}
        for inst in b_instantiations:
            r_value = inst['R']
            b_value = inst['B']
            r_label = ['pass', 'fail', 'no_results'][r_value]
            b_label = 'buy' if b_value == 0 else 'not_buy'
            b_policy[r_label] = b_label
        
        optimal_decisions = {
            'T': t_decision,
            'B': b_policy
        }
        
    finally:
        # Restore original values
        influence_diagram.utility(U)[{"T": "do", "B": "buy"}] = original_utilities["do_buy"]
        influence_diagram.utility(U)[{"T": "do", "B": "not_buy"}] = original_utilities["do_not_buy"]
        influence_diagram.utility(U)[{"T": "not_do", "B": "buy"}] = original_utilities["not_do_buy"]
        influence_diagram.utility(U)[{"T": "not_do", "B": "not_buy"}] = original_utilities["not_do_not_buy"]
    
    return meu, optimal_decisions


def update_utility_and_calculate_meu(u_table_modified):
    """
    Update utility values in the influence diagram and calculate MEU (backward compatibility)
    
    Parameters:
    -----------
    u_table_modified : pd.DataFrame
        Modified utility table
        
    Returns:
    --------
    float : Maximum Expected Utility
    """
    meu, _ = update_utility_and_calculate_meu_with_decisions(u_table_modified)
    return meu


def analyze_utility_sensitivity(u_table, baseline_meu, variation_amount=10, variation_type='percentage'):
    """
    Analyze sensitivity of utility values for tornado diagram
    
    Parameters:
    -----------
    u_table : pd.DataFrame
        Utility table
    baseline_meu : float
        Baseline maximum expected utility
    variation_amount : float
        Amount of variation to apply (percentage for 'percentage' type, absolute value for 'constant' type)
    variation_type : str
        Type of variation: 'percentage' or 'constant'
        
    Returns:
    --------
    list : Tornado data for utility variables
    """
    if variation_type == 'percentage':
        return analyze_utility_sensitivity_percentage(u_table, baseline_meu, variation_amount)
    elif variation_type == 'constant':
        return analyze_utility_sensitivity_constant(u_table, baseline_meu, variation_amount)
    else:
        raise ValueError("variation_type must be either 'percentage' or 'constant'")


def analyze_utility_sensitivity_percentage(u_table, baseline_meu, variation_percent=10):
    """
    Analyze sensitivity of utility values using percentage variations for tornado diagram
    
    Parameters:
    -----------
    u_table : pd.DataFrame
        Utility table
    baseline_meu : float
        Baseline maximum expected utility
    variation_percent : float
        Percentage variation to apply (e.g., 10 for ±10%)
        
    Returns:
    --------
    list : Tornado data for utility variables
    """
    tornado_data = []
    variation_factor = variation_percent / 100
    
    print(f"\nAnalyzing Utility Values (±{variation_percent}% variation)...")
    
    # Define utility scenarios to analyze
    utility_scenarios = [
        (0, 'U(do, buy, high)'),
        (1, 'U(do, buy, medium)'),
        (2, 'U(do, buy, low)'),
        (3, 'U(do, not_buy)'),
        (4, 'U(not_do, buy, high)'),
        (5, 'U(not_do, buy, medium)'),
        (6, 'U(not_do, buy, low)'),
        (7, 'U(not_do, not_buy)')
    ]
    
    for idx, scenario_name in utility_scenarios:
        # Create modified utility tables
        u_low = u_table.copy()
        u_high = u_table.copy()
        
        # Ensure the utility column is float type to avoid dtype warnings
        u_low = u_low.astype({u_low.columns[3]: float})
        u_high = u_high.astype({u_high.columns[3]: float})
        
        original_utility = u_table.iloc[idx, 3]
        
        # Apply percentage variation
        u_low.iloc[idx, 3] = original_utility * (1 - variation_factor)
        u_high.iloc[idx, 3] = original_utility * (1 + variation_factor)
        
        # Calculate expected utilities and optimal decisions
        meu_low, decisions_low = update_utility_and_calculate_meu_with_decisions(u_low)
        meu_high, decisions_high = update_utility_and_calculate_meu_with_decisions(u_high)
        
        impact_low = meu_low - baseline_meu
        impact_high = meu_high - baseline_meu
        total_range = abs(meu_high - meu_low)
        
        tornado_data.append({
            'variable': scenario_name,
            'low_impact': impact_low,
            'high_impact': impact_high,
            'total_range': total_range,
            'original_value': original_utility,
            'meu_low': meu_low,
            'meu_high': meu_high,
            'decisions_low': decisions_low,
            'decisions_high': decisions_high,
            'variation_type': 'percentage',
            'variation_amount': variation_percent
        })
        
        # Calculate the actual utility value changes
        low_utility_value = original_utility * (1 - variation_factor)
        high_utility_value = original_utility * (1 + variation_factor)
        low_change = low_utility_value - original_utility
        high_change = high_utility_value - original_utility
        
        print(f"  {scenario_name}: {original_utility:.0f} → Low={low_utility_value:.0f} ({low_change:+.0f}), High={high_utility_value:.0f} ({high_change:+.0f})")
    
    return tornado_data


def analyze_utility_sensitivity_constant(u_table, baseline_meu, variation_amount=50):
    """
    Analyze sensitivity of utility values using constant value variations for tornado diagram
    
    Parameters:
    -----------
    u_table : pd.DataFrame
        Utility table
    baseline_meu : float
        Baseline maximum expected utility
    variation_amount : float
        Constant amount to add/subtract (e.g., 50 for ±50 units)
        
    Returns:
    --------
    list : Tornado data for utility variables
    """
    tornado_data = []
    
    print(f"\nAnalyzing Utility Values (±{variation_amount} constant variation)...")
    
    # Define utility scenarios to analyze
    utility_scenarios = [
        (0, 'U(do, buy, high)'),
        (1, 'U(do, buy, medium)'),
        (2, 'U(do, buy, low)'),
        (3, 'U(do, not_buy)'),
        (4, 'U(not_do, buy, high)'),
        (5, 'U(not_do, buy, medium)'),
        (6, 'U(not_do, buy, low)'),
        (7, 'U(not_do, not_buy)')
    ]
    
    for idx, scenario_name in utility_scenarios:
        # Create modified utility tables
        u_low = u_table.copy()
        u_high = u_table.copy()
        
        # Ensure the utility column is float type to avoid dtype warnings
        u_low = u_low.astype({u_low.columns[3]: float})
        u_high = u_high.astype({u_high.columns[3]: float})
        
        original_utility = u_table.iloc[idx, 3]
        
        # Apply constant variation
        u_low.iloc[idx, 3] = original_utility - variation_amount
        u_high.iloc[idx, 3] = original_utility + variation_amount
        
        # Calculate expected utilities and optimal decisions
        meu_low, decisions_low = update_utility_and_calculate_meu_with_decisions(u_low)
        meu_high, decisions_high = update_utility_and_calculate_meu_with_decisions(u_high)
        
        impact_low = meu_low - baseline_meu
        impact_high = meu_high - baseline_meu
        total_range = abs(meu_high - meu_low)
        
        tornado_data.append({
            'variable': scenario_name,
            'low_impact': impact_low,
            'high_impact': impact_high,
            'total_range': total_range,
            'original_value': original_utility,
            'meu_low': meu_low,
            'meu_high': meu_high,
            'decisions_low': decisions_low,
            'decisions_high': decisions_high,
            'variation_type': 'constant',
            'variation_amount': variation_amount
        })
        
        # Calculate the actual utility value changes
        low_utility_value = original_utility - variation_amount
        high_utility_value = original_utility + variation_amount
        low_change = low_utility_value - original_utility
        high_change = high_utility_value - original_utility
        
        print(f"  {scenario_name}: {original_utility:.0f} → Low={low_utility_value:.0f} ({low_change:+.0f}), High={high_utility_value:.0f} ({high_change:+.0f})")
    
    return tornado_data


def create_utility_tornado_plot(tornado_data, baseline_meu, variation_amount=10, variation_type='percentage',
                               title="Utility Tornado Diagram", width=800, height=600):
    """
    Create tornado diagram using Plotly for utility sensitivity analysis
    
    Parameters:
    -----------
    tornado_data : list
        List of tornado data dictionaries
    baseline_meu : float
        Baseline expected utility
    variation_amount : float
        Amount of variation used (percentage or constant value)
    variation_type : str
        Type of variation: 'percentage' or 'constant'
    title : str
        Plot title
    width : int
        Plot width in pixels
    height : int
        Plot height in pixels
        
    Returns:
    --------
    plotly.graph_objects.Figure : The tornado plot figure
    """
    # Sort by total range (largest impact first)
    sorted_data = sorted(tornado_data, key=lambda x: x['total_range'], reverse=True)
    
    # Prepare data for plotting
    variables = [item['variable'] for item in sorted_data]
    low_impacts = [item['low_impact'] for item in sorted_data]
    high_impacts = [item['high_impact'] for item in sorted_data]
    original_values = [item['original_value'] for item in sorted_data]
    
    # Helper function to format decisions for hover text
    def format_decisions(decisions):
        t_decision = decisions['T']
        b_policy = decisions['B']
        b_text = ", ".join([f"{r}: {b}" for r, b in b_policy.items()])
        return f"T: {t_decision}, B: [{b_text}]"
    
    # Create the figure
    fig = go.Figure()
    
    # Add bars for low and high impacts
    for i, item in enumerate(sorted_data):
        var = item['variable']
        low = item['low_impact']
        high = item['high_impact']
        orig_val = item['original_value']
        meu_low = item['meu_low']
        meu_high = item['meu_high']
        decisions_low = item['decisions_low']
        decisions_high = item['decisions_high']
        
        # Determine which is left (negative) and right (positive)
        left_val = min(low, high)
        right_val = max(low, high)
        
        # Determine which scenario corresponds to left and right
        if low < high:
            left_meu = meu_low
            right_meu = meu_high
            left_decisions = decisions_low
            right_decisions = decisions_high
            left_label = "Low Scenario"
            right_label = "High Scenario"
        else:
            left_meu = meu_high
            right_meu = meu_low
            left_decisions = decisions_high
            right_decisions = decisions_low
            left_label = "High Scenario"
            right_label = "Low Scenario"
        
        # Create hover text for left bar
        left_hover_text = f"""
        Variable: {var}<br>
        {left_label}<br>
        MEU: {left_meu:.2f}<br>
        Impact: {left_val:+.2f}<br>
        Optimal Decisions: {format_decisions(left_decisions)}
        """
        
        # Create hover text for right bar  
        right_hover_text = f"""
        Variable: {var}<br>
        {right_label}<br>
        MEU: {right_meu:.2f}<br>
        Impact: {right_val:+.2f}<br>
        Optimal Decisions: {format_decisions(right_decisions)}
        """
        
        # Add left bar (from 0 to left_val) - no hover
        fig.add_trace(go.Bar(
            y=[var],
            x=[left_val],
            orientation='h',
            name='Lower Impact' if i == 0 else '',
            marker=dict(color='lightcoral', line=dict(color='black', width=0.5)),
            showlegend=i == 0,
            offsetgroup=1,
            width=0.6,
            hoverinfo='skip'
        ))
        
        # Add right bar (from 0 to right_val) - no hover
        fig.add_trace(go.Bar(
            y=[var],
            x=[right_val],
            orientation='h',
            name='Higher Impact' if i == 0 else '',
            marker=dict(color='lightblue', line=dict(color='black', width=0.5)),
            showlegend=i == 0,
            offsetgroup=1,
            width=0.6,
            hoverinfo='skip'
        ))
        
        # Add invisible scatter points for hover on left side
        fig.add_trace(go.Scatter(
            x=[left_val/2],
            y=[var],
            mode='markers',
            marker=dict(size=0.1, color='rgba(0,0,0,0)'),
            hovertemplate=left_hover_text + '<extra></extra>',
            hoverlabel=dict(bgcolor='lightcoral', bordercolor='darkred', font=dict(color='black')),
            showlegend=False,
            name=''
        ))
        
        # Add invisible scatter points for hover on right side
        fig.add_trace(go.Scatter(
            x=[right_val/2],
            y=[var],
            mode='markers',
            marker=dict(size=0.1, color='rgba(0,0,0,0)'),
            hovertemplate=right_hover_text + '<extra></extra>',
            hoverlabel=dict(bgcolor='lightblue', bordercolor='darkblue', font=dict(color='black')),
            showlegend=False,
            name=''
        ))
        
        # Add text annotations for impact values
        fig.add_annotation(
            x=left_val - (abs(left_val) * 0.1 if left_val != 0 else 1),
            y=var,
            text=f'{left_val:.1f}',
            showarrow=False,
            font=dict(size=10),
            xanchor='right' if left_val < 0 else 'left'
        )
        
        fig.add_annotation(
            x=right_val + (abs(right_val) * 0.1 if right_val != 0 else 1),
            y=var,
            text=f'{right_val:.1f}',
            showarrow=False,
            font=dict(size=10),
            xanchor='left' if right_val > 0 else 'right'
        )
    
    # Add vertical line at x=0
    fig.add_vline(x=0, line=dict(color='black', width=2))
    
    # Create subtitle based on variation type
    if variation_type == 'percentage':
        subtitle = f'Baseline Expected Utility: {baseline_meu:.2f} | Utility values varied by ±{variation_amount}%'
    else:  # constant
        subtitle = f'Baseline Expected Utility: {baseline_meu:.2f} | Utility values varied by ±{variation_amount} units'
    
    # Update layout
    fig.update_layout(
        title=dict(
            text=f'{title}<br><sub>{subtitle}</sub>',
            font=dict(size=16)
        ),
        xaxis=dict(
            title='Change in Expected Utility from Baseline',
            gridcolor='lightgray',
            gridwidth=0.5,
            zeroline=True,
            zerolinecolor='black',
            zerolinewidth=2
        ),
        yaxis=dict(
            title='Utility Variables',
            categoryorder='array',
            categoryarray=variables[::-1]  # Reverse to show highest impact at top
        ),
        width=width,
        height=height,
        bargap=0.1,
        bargroupgap=0,
        hovermode='closest',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    return fig


def generate_utility_tornado_data(u_table, baseline_meu, variation_amount=10, variation_type='percentage'):
    """
    Generate tornado analysis data for utility sensitivity analysis
    
    Parameters:
    -----------
    u_table : pd.DataFrame
        Utility table
    baseline_meu : float
        Baseline maximum expected utility
    variation_amount : float
        Amount of variation to apply (percentage for 'percentage' type, absolute value for 'constant' type)
    variation_type : str
        Type of variation: 'percentage' or 'constant'
        
    Returns:
    --------
    dict : Summary of results including baseline EU and variable impacts
    """
    print(f"Baseline Expected Utility: {baseline_meu:.2f}")
    
    # Analyze utility sensitivity
    tornado_data = analyze_utility_sensitivity(u_table, baseline_meu, variation_amount, variation_type)
    
    # Sort by total impact range (largest first)
    tornado_data.sort(key=lambda x: x['total_range'], reverse=True)
    
    return {
        'baseline_meu': baseline_meu,
        'tornado_data': tornado_data,
        'top_variable': tornado_data[0]['variable'] if tornado_data else None,
        'variation_amount': variation_amount,
        'variation_type': variation_type
    }


def plot_utility_tornado_diagram(tornado_results, title="Utility Tornado Diagram", 
                                width=800, height=600, show_plot=True):
    """
    Create and display tornado diagram from pre-computed tornado data
    
    Parameters:
    -----------
    tornado_results : dict
        Results from generate_utility_tornado_data()
    title : str
        Title for the plot
    width : int
        Plot width in pixels
    height : int
        Plot height in pixels
    show_plot : bool
        Whether to display the plot
        
    Returns:
    --------
    plotly.graph_objects.Figure : The tornado plot figure
    """
    # Extract data from results
    tornado_data = tornado_results['tornado_data']
    baseline_meu = tornado_results['baseline_meu']
    variation_amount = tornado_results['variation_amount']
    variation_type = tornado_results['variation_type']
    
    # Create the tornado plot
    fig = create_utility_tornado_plot(tornado_data, baseline_meu, variation_amount, variation_type, title, width, height)
    
    if show_plot:
        fig.show()
    
    return fig

In [8]:
# Generate tornado analysis data with constant variation
tornado_results_constant = generate_utility_tornado_data(
    u_table=u_table,
    baseline_meu=baseline_meu,
    variation_amount=100,  # ±100 units
    variation_type='constant'
)

print(f"\nMost sensitive utility variable (constant variation): {tornado_results_constant['top_variable']}")
print(f"Total variables analyzed: {len(tornado_results_constant['tornado_data'])}")

# Create and display the tornado plot for constant
fig_constant = plot_utility_tornado_diagram(
    tornado_results=tornado_results_constant,
    title="Oil Field Purchase: Utility Sensitivity Analysis (Constant Variation)",
    show_plot=False
)

print("\nDisplaying constant variation plot...")
fig_constant.show()

Baseline Expected Utility: 751.00

Analyzing Utility Values (±100 constant variation)...
  U(do, buy, high): 1250 → Low=1150 (-100), High=1350 (+100)
  U(do, buy, medium): 630 → Low=530 (-100), High=730 (+100)
  U(do, buy, low): 0 → Low=-100 (-100), High=100 (+100)
  U(do, not_buy): 350 → Low=250 (-100), High=450 (+100)
  U(not_do, buy, high): 1280 → Low=1180 (-100), High=1380 (+100)
  U(not_do, buy, medium): 660 → Low=560 (-100), High=760 (+100)
  U(not_do, buy, low): 30 → Low=-70 (-100), High=130 (+100)
  U(not_do, not_buy): 380 → Low=280 (-100), High=480 (+100)

Most sensitive utility variable (constant variation): U(not_do, buy, medium)
Total variables analyzed: 8

Displaying constant variation plot...


## 3 - Tornado analysis for test cost

In [9]:
def apply_test_cost_variation(u_table, test_cost_variation):
    """
    Apply test cost variation to utility table
    
    Parameters:
    -----------
    u_table : pd.DataFrame
        Original utility table
    test_cost_variation : float
        Amount to subtract from utilities when T='do' (positive means higher cost)
        
    Returns:
    --------
    pd.DataFrame : Modified utility table with test cost applied
    """
    u_modified = u_table.copy()
    u_modified = u_modified.astype({u_modified.columns[3]: float})
    
    # Apply test cost variation to all "do" cases
    # Indices 0, 1, 2, 3 correspond to 'do' cases in the utility table
    do_indices = u_modified[u_modified['T'] == 'do'].index
    u_modified.loc[do_indices, 'U'] = u_modified.loc[do_indices, 'U'] - test_cost_variation
    
    return u_modified


def update_utility_and_calculate_meu_with_decisions(u_table_modified):
    """
    Update utility values in the influence diagram and calculate MEU with optimal decisions
    
    Parameters:
    -----------
    u_table_modified : pd.DataFrame
        Modified utility table
        
    Returns:
    --------
    tuple : (MEU, optimal_decisions_dict)
    """
    # Store original utility values
    original_utilities = {}
    original_utilities["do_buy"] = influence_diagram.utility(U)[{"T": "do", "B": "buy"}].copy()
    original_utilities["do_not_buy"] = influence_diagram.utility(U)[{"T": "do", "B": "not_buy"}].copy()
    original_utilities["not_do_buy"] = influence_diagram.utility(U)[{"T": "not_do", "B": "buy"}].copy()
    original_utilities["not_do_not_buy"] = influence_diagram.utility(U)[{"T": "not_do", "B": "not_buy"}].copy()
    
    try:
        # Update utility values
        influence_diagram.utility(U)[{"T": "do", "B": "buy"}] = np.array(
            [u_table_modified.iloc[0, 3], u_table_modified.iloc[1, 3], u_table_modified.iloc[2, 3]])[:, np.newaxis]
        influence_diagram.utility(U)[{"T": "do", "B": "not_buy"}] = np.array(
            [u_table_modified.iloc[3, 3]] * 3)[:, np.newaxis]
        influence_diagram.utility(U)[{"T": "not_do", "B": "buy"}] = np.array(
            [u_table_modified.iloc[4, 3], u_table_modified.iloc[5, 3], u_table_modified.iloc[6, 3]])[:, np.newaxis]
        influence_diagram.utility(U)[{"T": "not_do", "B": "not_buy"}] = np.array(
            [u_table_modified.iloc[7, 3]] * 3)[:, np.newaxis]
        
        # Calculate MEU and optimal decisions
        temp_inference = grum.ShaferShenoyLIMIDInference(influence_diagram)
        temp_inference.makeInference()
        meu_result = temp_inference.MEU()
        meu = meu_result['mean']
        
        # Get optimal decisions
        optimal_T = temp_inference.optimalDecision("T").argmax()
        optimal_B_tensor = temp_inference.optimalDecision("B")
        
        # For T decision (simple decision)
        t_decision = 'do' if optimal_T[0][0]['T'] == 0 else 'not_do'
        
        # For B decision (depends on R), extract the policy
        b_instantiations, _ = optimal_B_tensor.argmax()
        b_policy = {}
        for inst in b_instantiations:
            r_value = inst['R']
            b_value = inst['B']
            r_label = ['pass', 'fail', 'no_results'][r_value]
            b_label = 'buy' if b_value == 0 else 'not_buy'
            b_policy[r_label] = b_label
        
        optimal_decisions = {
            'T': t_decision,
            'B': b_policy
        }
        
    finally:
        # Restore original values
        influence_diagram.utility(U)[{"T": "do", "B": "buy"}] = original_utilities["do_buy"]
        influence_diagram.utility(U)[{"T": "do", "B": "not_buy"}] = original_utilities["do_not_buy"]
        influence_diagram.utility(U)[{"T": "not_do", "B": "buy"}] = original_utilities["not_do_buy"]
        influence_diagram.utility(U)[{"T": "not_do", "B": "not_buy"}] = original_utilities["not_do_not_buy"]
    
    return meu, optimal_decisions


def analyze_test_cost_sensitivity(u_table, baseline_meu, cost_variations):
    """
    Analyze sensitivity of test cost for tornado diagram
    
    Parameters:
    -----------
    u_table : pd.DataFrame
        Original utility table
    baseline_meu : float
        Baseline maximum expected utility
    cost_variations : list
        List of test cost variations to analyze (e.g., [-50, -25, 0, 25, 50, 100])
        
    Returns:
    --------
    list : Tornado data for test cost variable
    """
    tornado_data = []
    
    print(f"\nAnalyzing Test Cost Sensitivity...")
    print(f"Cost variations to test: {cost_variations}")
    
    for cost_variation in cost_variations:
        # Apply test cost variation
        u_modified = apply_test_cost_variation(u_table, cost_variation)
        
        # Calculate MEU and optimal decisions
        meu, decisions = update_utility_and_calculate_meu_with_decisions(u_modified)
        
        impact = meu - baseline_meu
        
        tornado_data.append({
            'cost_variation': cost_variation,
            'meu': meu,
            'impact': impact,
            'decisions': decisions
        })
        
        print(f"  Test cost variation: {cost_variation:+.0f} → MEU: {meu:.2f} (Impact: {impact:+.2f})")
    
    return tornado_data


def analyze_test_cost_tornado(u_table, baseline_meu, cost_variation=50):
    """
    Analyze test cost sensitivity for tornado diagram with a single cost variation
    
    Parameters:
    -----------
    u_table : pd.DataFrame
        Original utility table
    baseline_meu : float
        Baseline maximum expected utility
    cost_variation : float
        Single cost variation value to analyze (will test both +cost_variation and -cost_variation)
        
    Returns:
    --------
    dict : Tornado analysis results for test cost
    """
    # Test both positive and negative variations
    cost_variations = [-cost_variation, cost_variation]
    
    print(f"Test Cost Tornado Analysis")
    print(f"Baseline MEU: {baseline_meu:.2f}")
    print(f"Cost variation: ±{cost_variation}")
    
    # Analyze sensitivity for both variations
    sensitivity_data = analyze_test_cost_sensitivity(u_table, baseline_meu, cost_variations)
    
    # Get low and high scenarios
    low_data = sensitivity_data[0]   # -cost_variation
    high_data = sensitivity_data[1]  # +cost_variation
    
    # Create tornado data in expected format
    tornado_data = [{
        'variable': 'Test Cost',
        'low_impact': low_data['impact'],
        'high_impact': high_data['impact'],
        'total_range': abs(high_data['impact'] - low_data['impact']),
        'low_cost_variation': low_data['cost_variation'],
        'high_cost_variation': high_data['cost_variation'],
        'meu_low': low_data['meu'],
        'meu_high': high_data['meu'],
        'decisions_low': low_data['decisions'],
        'decisions_high': high_data['decisions'],
        'cost_variation': cost_variation,
        'sensitivity_data': sensitivity_data
    }]
    
    return {
        'baseline_meu': baseline_meu,
        'tornado_data': tornado_data,
        'sensitivity_data': sensitivity_data,
        'cost_variation': cost_variation
    }


def create_test_cost_tornado_plot(tornado_results, title="Test Cost Tornado Diagram", 
                                 width=800, height=400):
    """
    Create tornado diagram for test cost sensitivity analysis
    
    Parameters:
    -----------
    tornado_results : dict
        Results from analyze_test_cost_tornado()
    title : str
        Plot title
    width : int
        Plot width in pixels
    height : int
        Plot height in pixels
        
    Returns:
    --------
    plotly.graph_objects.Figure : The tornado plot figure
    """
    tornado_data = tornado_results['tornado_data'][0]  # Only one variable
    baseline_meu = tornado_results['baseline_meu']
    cost_variation = tornado_results['cost_variation']
    
    # Extract data
    variable = tornado_data['variable']
    low_impact = tornado_data['low_impact']
    high_impact = tornado_data['high_impact']
    low_cost = tornado_data['low_cost_variation']
    high_cost = tornado_data['high_cost_variation']
    meu_low = tornado_data['meu_low']
    meu_high = tornado_data['meu_high']
    decisions_low = tornado_data['decisions_low']
    decisions_high = tornado_data['decisions_high']
    
    # Helper function to format decisions for hover text
    def format_decisions(decisions):
        t_decision = decisions['T']
        b_policy = decisions['B']
        b_text = ", ".join([f"{r}: {b}" for r, b in b_policy.items()])
        return f"T: {t_decision}, B: [{b_text}]"
    
    # Create the figure
    fig = go.Figure()
    
    # Determine which is left (negative) and right (positive)
    left_val = min(low_impact, high_impact)
    right_val = max(low_impact, high_impact)
    
    # Determine which scenario corresponds to left and right
    if low_impact < high_impact:
        left_meu = meu_low
        right_meu = meu_high
        left_decisions = decisions_low
        right_decisions = decisions_high
        left_cost = low_cost
        right_cost = high_cost
        left_label = f"Low Cost (Cost: {left_cost:+.0f})"
        right_label = f"High Cost (Cost: {right_cost:+.0f})"
    else:
        left_meu = meu_high
        right_meu = meu_low
        left_decisions = decisions_high
        right_decisions = decisions_low
        left_cost = high_cost
        right_cost = low_cost
        left_label = f"High Cost (Cost: {left_cost:+.0f})"
        right_label = f"Low Cost (Cost: {right_cost:+.0f})"
    
    # Create hover text
    left_hover_text = f"""
    Variable: {variable}<br>
    {left_label}<br>
    MEU: {left_meu:.2f}<br>
    Impact: {left_val:+.2f}<br>
    Optimal Decisions: {format_decisions(left_decisions)}
    """
    
    right_hover_text = f"""
    Variable: {variable}<br>
    {right_label}<br>
    MEU: {right_meu:.2f}<br>
    Impact: {right_val:+.2f}<br>
    Optimal Decisions: {format_decisions(right_decisions)}
    """
    
    # Add left bar
    fig.add_trace(go.Bar(
        y=[variable],
        x=[left_val],
        orientation='h',
        name='Lower Impact',
        marker=dict(color='lightcoral', line=dict(color='black', width=0.5)),
        offsetgroup=1,
        width=0.6,
        hoverinfo='skip'
    ))
    
    # Add right bar
    fig.add_trace(go.Bar(
        y=[variable],
        x=[right_val],
        orientation='h',
        name='Higher Impact',
        marker=dict(color='lightblue', line=dict(color='black', width=0.5)),
        offsetgroup=1,
        width=0.6,
        hoverinfo='skip'
    ))
    
    # Add invisible scatter points for hover
    fig.add_trace(go.Scatter(
        x=[left_val/2],
        y=[variable],
        mode='markers',
        marker=dict(size=0.1, color='rgba(0,0,0,0)'),
        hovertemplate=left_hover_text + '<extra></extra>',
        showlegend=False,
        name=''
    ))
    
    fig.add_trace(go.Scatter(
        x=[right_val/2],
        y=[variable],
        mode='markers',
        marker=dict(size=0.1, color='rgba(0,0,0,0)'),
        hovertemplate=right_hover_text + '<extra></extra>',
        showlegend=False,
        name=''
    ))
    
    # Add text annotations for impact values
    fig.add_annotation(
        x=left_val - (abs(left_val) * 0.1 if left_val != 0 else 1),
        y=variable,
        text=f'{left_val:.1f}',
        showarrow=False,
        font=dict(size=12),
        xanchor='right' if left_val < 0 else 'left'
    )
    
    fig.add_annotation(
        x=right_val + (abs(right_val) * 0.1 if right_val != 0 else 1),
        y=variable,
        text=f'{right_val:.1f}',
        showarrow=False,
        font=dict(size=12),
        xanchor='left' if right_val > 0 else 'right'
    )
    
    # Add vertical line at x=0
    fig.add_vline(x=0, line=dict(color='black', width=2))
    
    # Create subtitle
    subtitle = f'Baseline Expected Utility: {baseline_meu:.2f} | Test cost varied by ±{cost_variation} units'
    
    # Update layout
    fig.update_layout(
        title=dict(
            text=f'{title}<br><sub>{subtitle}</sub>',
            font=dict(size=16)
        ),
        xaxis=dict(
            title='Change in Expected Utility from Baseline',
            gridcolor='lightgray',
            gridwidth=0.5,
            zeroline=True,
            zerolinecolor='black',
            zerolinewidth=2
        ),
        yaxis=dict(
            title='Variable',
            showticklabels=True
        ),
        width=width,
        height=height,
        bargap=0.1,
        bargroupgap=0,
        hovermode='closest',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    return fig


def plot_test_cost_sensitivity_curve(tornado_results, title="Test Cost Sensitivity Curve",
                                    width=800, height=500):
    """
    Create a sensitivity curve showing how MEU changes with test cost
    
    Parameters:
    -----------
    tornado_results : dict
        Results from analyze_test_cost_tornado()
    title : str
        Plot title
    width : int
        Plot width in pixels
    height : int
        Plot height in pixels
        
    Returns:
    --------
    plotly.graph_objects.Figure : The sensitivity curve figure
    """
    sensitivity_data = tornado_results['sensitivity_data']
    baseline_meu = tornado_results['baseline_meu']
    
    # Extract data for plotting
    cost_variations = [item['cost_variation'] for item in sensitivity_data]
    meus = [item['meu'] for item in sensitivity_data]
    impacts = [item['impact'] for item in sensitivity_data]
    
    # Create figure with secondary y-axis
    fig = go.Figure()
    
    # Add MEU curve
    fig.add_trace(go.Scatter(
        x=cost_variations,
        y=meus,
        mode='lines+markers',
        name='MEU',
        line=dict(color='blue', width=2),
        marker=dict(size=6),
        hovertemplate='Cost Variation: %{x:+.0f}<br>MEU: %{y:.2f}<extra></extra>'
    ))
    
    # Add baseline line
    fig.add_hline(
        y=baseline_meu,
        line=dict(color='red', width=2, dash='dash'),
        annotation_text=f'Baseline MEU: {baseline_meu:.2f}',
        annotation_position="top right"
    )
    
    # Add vertical line at x=0 (no cost change)
    fig.add_vline(
        x=0,
        line=dict(color='gray', width=1, dash='dot'),
        annotation_text='No Cost Change',
        annotation_position="top"
    )
    
    # Update layout
    fig.update_layout(
        title=dict(
            text=title,
            font=dict(size=16)
        ),
        xaxis=dict(
            title='Test Cost Variation (units)',
            gridcolor='lightgray',
            gridwidth=0.5,
            zeroline=True,
            zerolinecolor='gray',
            zerolinewidth=1
        ),
        yaxis=dict(
            title='Maximum Expected Utility',
            gridcolor='lightgray',
            gridwidth=0.5
        ),
        width=width,
        height=height,
        hovermode='x unified',
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    return fig

def analyze_test_cost_breakeven(u_table, baseline_meu, max_search_cost=200, tolerance=0.01):
    """
    Find the breakeven test cost where the optimal decision changes
    
    Parameters:
    -----------
    u_table : pd.DataFrame
        Original utility table
    baseline_meu : float
        Baseline maximum expected utility
    max_search_cost : float
        Maximum cost to search up to
    tolerance : float
        Tolerance for MEU comparison
        
    Returns:
    --------
    dict : Breakeven analysis results
    """
    print(f"\nFinding test cost breakeven point...")
    
    # Get baseline optimal decision
    _, baseline_decisions = update_utility_and_calculate_meu_with_decisions(u_table)
    baseline_t_decision = baseline_decisions['T']
    
    print(f"Baseline optimal T decision: {baseline_t_decision}")
    
    # Search for breakeven point
    cost_increment = 1.0
    current_cost = 0.0
    breakeven_cost = None
    
    while current_cost <= max_search_cost:
        u_modified = apply_test_cost_variation(u_table, current_cost)
        meu, decisions = update_utility_and_calculate_meu_with_decisions(u_modified)
        
        if decisions['T'] != baseline_t_decision:
            breakeven_cost = current_cost
            breakeven_meu = meu
            breakeven_decisions = decisions
            break
            
        current_cost += cost_increment
    
    if breakeven_cost is not None:
        print(f"Breakeven point found at test cost: +{breakeven_cost:.0f}")
        print(f"Optimal decision changes from '{baseline_t_decision}' to '{breakeven_decisions['T']}'")
        print(f"MEU at breakeven: {breakeven_meu:.2f}")
        
        return {
            'breakeven_cost': breakeven_cost,
            'baseline_t_decision': baseline_t_decision,
            'breakeven_t_decision': breakeven_decisions['T'],
            'breakeven_meu': breakeven_meu,
            'breakeven_decisions': breakeven_decisions
        }
    else:
        print(f"No breakeven point found within search range (0 to {max_search_cost})")
        return None


In [10]:
tornado_results = analyze_test_cost_tornado(
        u_table=u_table,
        baseline_meu=baseline_meu,
        cost_variation=25
    )
    
# Create and show tornado plot
tornado_fig = create_test_cost_tornado_plot(tornado_results)
tornado_fig.show()

Test Cost Tornado Analysis
Baseline MEU: 751.00
Cost variation: ±25

Analyzing Test Cost Sensitivity...
Cost variations to test: [-25, 25]
  Test cost variation: -25 → MEU: 751.95 (Impact: +0.95)
  Test cost variation: +25 → MEU: 751.00 (Impact: +0.00)
